# DAIN Colab

*DAIN Colab, v1.4.0*

Based on the [original Colab file](https://github.com/baowenbo/DAIN/issues/44) by btahir. 

Enhancements by [Styler00Dollar](https://github.com/styler00dollar) aka "sudo rm -rf / --no-preserve-root#8353" on discord and [Alpha](https://github.com/AlphaGit), (Alpha#6137 on Discord). Please do not run this command in your linux terminal. It's rather meant as a joke.

[Styler00Dollar's fork](https://github.com/styler00dollar/DAIN) / [Alpha's fork](https://github.com/AlphaGit/DAIN)

A simple guide:
- Upload this ` .ipynb`  file to your Google Colab.
- Create a folder inside of Google Drive named "DAIN"
- Change the configurations in the next cell
- Run cells one by one

Stuff that should be improved:
- Alpha channel will be removed automatically and won't be added back. Anything related to alpha will be converted to black.
- Adding configuration to select speed
- Detect scenes to avoid interpolating scene-changes
- Auto-resume
- Copy `start_frame` - `end_frame` audio from original input to final output


In [0]:
################# Required Configurations ############################

#@markdown # Required Configuration
#@markdown Use the values in here to configure what you'd like DAIN to do.

#@markdown ## Input file
#@markdown Path (relative to the root of your Google Drive) to the input file. For instance, if you save your `example.mkv` file in your Google Drive, inside a `videos` folder, the path would be: `videos/example.mkv`. Currenly videos and gifs are supported.
INPUT_FILEPATH = "DAIN/input.mp4" #@param{type:"string"}

#@markdown ## Output file
#@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "DAIN/output.mp4" #@param{type:"string"}

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS = 60 #@param{type:"number"}

#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = '/content/DAIN/input_frames' #@param{type:"string"}

#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames' #@param{type:"string"}

#@markdown ## Seamless playback
#@markdown Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = False #@param{type:"boolean"}

#@markdown ## Resize hotfix
#@markdown DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1). Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
#@markdown
#@markdown This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP uses as a fix for this problem, but the original does show such behaviour with the default test images. More advanced users can change the interpolation method. The methods cv2.INTER_CUBIC and cv2.INTER_LANCZOS4 are recommended. The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True #@param{type:"boolean"}

#@markdown ## Auto-remove PNG directory
#@markdown Auto-delete output PNG dir after ffmpeg video creation. Set this to `False` if you want to keep the PNG files.
AUTO_REMOVE = True #@param{type:"boolean"}

In [0]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

In [0]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

# Install dependencies.

This next step may take somewhere between 15-20 minutes. Run this only once at startup.

Look for the "Finished installing dependencies"  message.

In [0]:
# Install known used versions of PyTorch and SciPy
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install scipy==1.1.0

In [0]:
# Clone DAIN sources
# TODO Bring back original repo
#!git clone https://github.com/baowenbo/DAIN /content/DAIN
%cd /content
!rm -rf DAIN
!git clone -b colab-compatibility --depth 1 https://github.com/AlphaGit/DAIN /content/DAIN
%cd /content/DAIN
!git log -1

In [0]:
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/DAIN/my_package/
!./build.sh
print("Building #1 done.")

In [0]:
# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")

In [0]:
# Downloading pre-trained model
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

In [0]:
!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
print("Finished installing dependencies.")

In [0]:
# Detecting FPS of input file.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

In [0]:
# ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

%shell ffmpeg -i '/content/DAIN/{filename}' '{FRAME_INPUT_DIR}/%05d.png'

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
from IPython.display import clear_output
clear_output()

pngs_generated_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS==True:
  pngs_generated_count += 1
  original = str(FRAME_INPUT_DIR)+"/00001.png"
  target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)

print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

# Checking if PNG do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

In [0]:
# Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

In [0]:
# Finding DAIN Frames, upscaling and cropping to match original
import numpy as np
%cd {FRAME_OUTPUT_DIR}
if(RESIZE_HOTFIX==True):
  images = []
  for filename in os.listdir(f'{FRAME_OUTPUT_DIR}'):
    img = cv2.imread(os.path.join(f'{FRAME_OUTPUT_DIR}',filename))
    part_filename = os.path.splitext(filename)
    if(part_filename[0].endswith('0')==False):
      dimension = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      cv2.imwrite(part_filename[0]+".png", crop)

%cd /content/DAIN

In [0]:
# Create video
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

In [0]:
if(AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*

In [0]:
# [Experimental] Create video with sound
# Only run this, if the original had sound.
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -i '/content/DAIN/{filename}' -acodec copy output-audio.aac
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i output-audio.aac -shortest '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

if (AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*
  !rm -rf output-audio.aac